In [6]:
import librosa
import numpy as np
import math
import separator
import soundfile as sf
from yt_dlp import YoutubeDL
from scipy import stats
import warnings

# librosaが出力する警告を非表示にする
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def my_round_int(number):
    """数値を四捨五入して整数にする"""
    return int((number * 2 + 1) // 2)

def hz_to_midi(hz):
    """周波数(Hz)をMIDIノートナンバーに変換する"""
    if hz is None or hz <= 0:
        return None
    midi = 12 * (math.log2(hz / 440)) + 69
    return my_round_int(midi)

def midi_to_simplified_note_name(midi_number):
    """MIDIノートナンバーをhi/mid/low形式の音名に変換する"""
    if not 0 <= midi_number <= 127:
        return "N/A"

    note_names_base = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    note_index = int(midi_number % 12)
    base_note = note_names_base[note_index]

    if midi_number <= 20:
        return "lowlowlow" + base_note
    elif midi_number <= 32:
        return "lowlow" + base_note
    elif midi_number <= 44:
        return "low" + base_note
    elif midi_number <= 56:
        return "mid1" + base_note
    elif midi_number <= 68:
        return "mid2" + base_note
    elif midi_number <= 80:
        return "hi" + base_note
    elif midi_number <= 92:
        return "hihi" + base_note
    elif midi_number <= 104:
        return "hihihi" + base_note
    else:
        return "hihihihi" + base_note

In [ ]:
def analyze_song_pitch(youtube_url):
    """
    YouTube URLからボーカルを分析し、最高音、最低音、中心音の情報をまとめて返す関数
    """
    # 1. 曲名の取得
    with YoutubeDL({'quiet': True, 'extract_flat': True}) as ydl:
        try:
            info = ydl.extract_info(youtube_url, download=False)
            title = info.get('title', 'downloaded_audio')
            print(f"「{title}」の音源を処理中...")
        except Exception as e:
            print(f"曲情報の取得中にエラー: {e}")
            return None

    # 2. ボーカル分離（pitch_track_test.ipynbのフローを忠実に再現）
    print("ボーカル分離処理を開始...")
    try:
        file = separator.download_and_separate_audio(youtube_url)
        reverb_removed_file = separator.revarb_remove(f"./separated_audio/{file}")
        harmony_removed_file = separator.harmony_remove(f"./separated_audio/{reverb_removed_file}")
        final_vocal_file_name = separator.noize_remove(f"./separated_audio/{harmony_removed_file}")
        final_vocal_path = f'./separated_audio/{final_vocal_file_name}'
    except Exception as e:
        print(f"ボーカル分離中にエラー: {e}")
        return None
    
    # 3. ピッチ分析
    try:
        print("ピッチ分析を開始...")
        audio, sr = librosa.load(final_vocal_path, sr=44100)
        normalized_audio = librosa.util.normalize(audio)
        
        intervals = librosa.effects.split(normalized_audio, top_db=30)
        if not intervals.any():
            print("エラー: 有効な音声区間が見つかりませんでした。")
            return None

        all_f0 = [f0 for start_i, end_i in intervals if (f0 := librosa.pyin(normalized_audio[start_i:end_i], fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C6'))[0]).any()]
        
        f0_combined = np.concatenate(all_f0) if all_f0 else np.array([])
        valid_f0 = f0_combined[~np.isnan(f0_combined)]

        if len(valid_f0) < 2: return None

        mean, std_dev = np.mean(valid_f0), np.std(valid_f0)
        f0_cleaned_hz = valid_f0[np.abs((valid_f0 - mean) / std_dev) < 2.0]
        if len(f0_cleaned_hz) == 0: return None

        f0_midi = [hz_to_midi(f) for f in f0_cleaned_hz if f is not None]
        if not f0_midi: return None

        # 4. 最高音、最低音、中心音（最頻値）を算出
        max_note = midi_to_simplified_note_name(hz_to_midi(np.max(f0_cleaned_hz)))
        min_note = midi_to_simplified_note_name(hz_to_midi(np.min(f0_cleaned_hz)))
        central_midi = int(stats.mode(f0_midi, keepdims=False).mode)
        central_note = midi_to_simplified_note_name(central_midi)
        
        print("分析完了。")
        return {
            "title": title, "max_note": max_note,
            "min_note": min_note, "central_midi": central_midi,
            "central_note": central_note
        }
    
    except Exception as e:
        print(f"ピッチ分析中にエラー: {e}")
        return None

In [ ]:
# ユーザーからのURL入力
original_url = input("原曲のYouTube URLを入力してください: ")
cover_url = input("カバー音源のYouTube URLを入力してください: ")

print("\n--- 原曲の分析を開始します ---")
original_results = analyze_song_pitch(original_url)

print("\n--- カバー音源の分析を開始します ---")
cover_results = analyze_song_pitch(cover_url)

# --- 最終結果の表示 ---
print("\n" + "="*17 + " キー比較 分析結果 " + "="*17)

if original_results:
    print(f"【原曲】: {original_results['title']}")
    print(f"  中心音: {original_results['central_note']}")
    print(f"  最高音: {original_results['max_note']}")
    print(f"  最低音: {original_results['min_note']}")
else:
    print("【原曲】の分析に失敗しました。")

print() # 改行

if cover_results:
    print(f"【カバー】: {cover_results['title']}")
    print(f"  中心音: {cover_results['central_note']}")
    print(f"  最高音: {cover_results['max_note']}")
    print(f"  最低音: {cover_results['min_note']}")
else:
    print("【カバー】の分析に失敗しました。")

print("\n" + "-"*48)

if original_results and cover_results:
    key_difference = cover_results['central_midi'] - original_results['central_midi']
    print(f"キーの差は{key_difference:+.0f}です。")
else:
    print("キーの差は分析できませんでした。")

print("-" * 48)


--- 原曲の分析を開始します ---


         player = https://www.youtube.com/s/player/e12fbea4/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


「yama - 春を告げる (Official Video)」の音源を処理中...
ボーカル分離処理を開始...
[youtube] Extracting URL: https://youtu.be/DC6JppqHkaM?si=RlAK52WzCAWKghlL
[youtube] DC6JppqHkaM: Downloading webpage
[youtube] DC6JppqHkaM: Downloading tv client config
[youtube] DC6JppqHkaM: Downloading tv player API JSON
[youtube] DC6JppqHkaM: Downloading ios player API JSON
[youtube] DC6JppqHkaM: Downloading player e12fbea4-main


         player = https://www.youtube.com/s/player/e12fbea4/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] DC6JppqHkaM: Downloading m3u8 information
[info] Testing format 234
[info] DC6JppqHkaM: Downloading 1 format(s): 234
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 34
[download] Destination: downloaded_audio\download.mp4
[download] 100% of    3.05MiB in 00:00:00 at 5.11MiB/s                   
[ExtractAudio] Destination: downloaded_audio\download.wav
Deleting original file downloaded_audio\download.mp4 (pass -k to keep)
[Metadata] Adding metadata to "downloaded_audio\download.wav"
[youtube] Extracting URL: https://youtu.be/DC6JppqHkaM?si=RlAK52WzCAWKghlL
[youtube] DC6JppqHkaM: Downloading webpage
[youtube] DC6JppqHkaM: Downloading tv client config
[youtube] DC6JppqHkaM: Downloading tv player API JSON
[youtube] DC6JppqHkaM: Downloading ios player API JSON


...

[youtube] B-knvTjLj8U: Downloading m3u8 information
[info] Testing format 234


2025-07-02 04:10:48,717 - INFO - separator - Separator version 0.34.1 instantiating with output_dir: ./separated_audio, output_format: WAV
2025-07-02 04:10:48,717 - INFO - separator - Using model directory from model_file_dir parameter: /tmp/audio-separator-models/
2025-07-02 04:10:48,718 - INFO - separator - Operating System: Windows 10.0.26100
2025-07-02 04:10:48,718 - INFO - separator - System: Windows Node: DESKTOP-TQH7KB4 Release: 11 Machine: AMD64 Proc: AMD64 Family 26 Model 68 Stepping 0, AuthenticAMD
2025-07-02 04:10:48,718 - INFO - separator - Python Version: 3.13.5
2025-07-02 04:10:48,718 - INFO - separator - PyTorch Version: 2.7.1+cu128
2025-07-02 04:10:48,737 - INFO - separator - FFmpeg installed: ffmpeg version 7.1.1-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
2025-07-02 04:10:48,741 - INFO - separator - ONNX Runtime GPU package installed with version: 1.22.0
2025-07-02 04:10:48,742 - INFO - separator - CUDA is available in Torch, setting Torch de

Downloaded audio for: 春を告げる / 葛葉【歌ってみた】
春を告げる___葛葉【歌ってみた】


2025-07-02 04:10:49,966 - INFO - mdxc_separator - MDXC Separator initialisation complete
2025-07-02 04:10:49,967 - INFO - separator - Load model duration: 00:00:01
2025-07-02 04:10:49,967 - INFO - separator - Processing file: ./downloaded_audio/download.wav
2025-07-02 04:10:49,968 - INFO - separator - Starting separation process for audio_file_path: ./downloaded_audio/download.wav
100%|██████████| 25/25 [00:13<00:00,  1.79it/s]
2025-07-02 04:11:04,100 - INFO - mdxc_separator - Saving vocals stem to Vocals.wav...
2025-07-02 04:11:04,112 - INFO - common_separator - Audio duration is 0.05 hours (197.42 seconds).
2025-07-02 04:11:04,113 - INFO - common_separator - Using pydub for writing.
2025-07-02 04:11:04,412 - INFO - common_separator - Clearing input audio file paths, sources and stems...
2025-07-02 04:11:04,448 - INFO - separator - Separation duration: 00:00:14
2025-07-02 04:11:04,450 - INFO - separator - Separator version 0.34.1 instantiating with output_dir: ./separated_audio, outpu

ピッチ分析を開始...
分析完了。中心音: G#2 (MIDI: 44)

           キー比較 分析結果
【原曲】: yama - 春を告げる (Official Video)
  中心音: F#3 (MIDI: 54)

【カバー】: 春を告げる / 葛葉【歌ってみた】
  中心音: G#2 (MIDI: 44)

----------------------------------------
キーの差: -10 (半音)
----------------------------------------

結論: カバー音源は原曲よりキーが【10】つ低いようです。
